# TIES581 Project work

Mikael Myyrä  
`mikael.b.myyra@jyu.fi`

In this document I implement and test Krylov subspace methods for
linear systems using NumPy, based on the descriptions of Saad (2003).

# Test problems and library setup

Using numpy for matrix utilities and scipy to read the Harwell-Boeing matrix format.

The matrices are picked from the Harwell-Boeing and FIDAP collections on
[Matrix Market](https://math.nist.gov/MatrixMarket/).
Two of them, ORSIRR1 and FIDAP36, are also used by Saad (2003).
FIDAP005 is a smaller problem that is helpful in early testing
because you can reasonably print and read it.

Following Saad (2003) chapter 3.7, the right-hand side of $Ax = b$ is generated
as $b = Ae$ where $e = (1,1,\dots,1)^T$, and the initial guess $x_0$
is a vector of random values. Saad does not specify the range
or distribution of random values, so I am assuming the conventional
uniform distribution in the range $[0, 1)$.

In [1]:
import numpy as np
import scipy as sp
from scipy.io import mmread

import math
import time
from dataclasses import dataclass
from typing import Callable
from typing import Union

def default_rhs(A) -> np.ndarray:
    return A * np.ones((A.shape[1],))

def random_guess(A) -> np.ndarray:
    return np.random.random_sample((A.shape[1],))

@dataclass
class Equation:
    name: str
    A: np.ndarray
    b: np.ndarray

    def residual(self, x: np.ndarray) -> np.ndarray:
        return self.b - self.A * x

TEST_EQUATIONS: list[Equation] = [
    Equation(name=name, A=mat, b=default_rhs(mat))
    for name, mat in [
        ("FIDAP005", mmread("test_matrices/fidap005.mtx")),
        ("FIDAP036", mmread("test_matrices/fidap036.mtx")),
        ("GR3030", mmread("test_matrices/gr_30_30.mtx")),
        ("ORSIRR1", mmread("test_matrices/orsirr_1.mtx")),
    ]
]

MAX_ITERATIONS = 300
EPSILON = 1e-7

@dataclass
class RunResult:
    ans: np.ndarray
    iterations: int

def test_run(method: Callable[[Equation, np.ndarray], np.ndarray], **kwargs):
    """Run the given method on all the test problems and print statistics."""

    # pretty-printing results as a table
    CELL_SIZE = 15
    def fmt_cell(x: Union[str, float]) -> str:
        text = f"{x:.3e}" if isinstance(x, float) else str(x)
        return text.center(CELL_SIZE)
    def print_row(cells: list[Union[str, float]]):
        print("|".join([fmt_cell(c) for c in cells]))

    headers = ["matrix", "size", "pre-residual", "post-residual", "iterations", "time (ms)"]
    print_row(headers)
    print_row(["-" * CELL_SIZE] * len(headers))
    for eq in TEST_EQUATIONS:
        x0 = random_guess(eq.A)
        initial_resid = np.linalg.norm(eq.residual(x0))
        start_time = time.perf_counter_ns()

        result = method(eq, x0, **kwargs)

        duration_ns = time.perf_counter_ns() - start_time
        duration_ms = duration_ns // 1000000
        resid = np.linalg.norm(eq.b - eq.A * result.ans)
        size = f"{eq.A.shape[0]} x {eq.A.shape[1]}"
        print_row([eq.name, size, initial_resid, resid, result.iterations, duration_ms])

np.set_printoptions(precision=5)

# Methods

## Full Orthogonalization Method (FOM)

FOM (or Arnoldi's Method) is an algorithm to approximately solve
a linear system $Ax = b$ using a Krylov subspace

$$
\mathcal{K}_m = \text{span}\{r_0, Ar_0, A^2r_0, \dots, A^{m-1}r_0\}
$$

where $m$ is the dimension of the subspace and $r_0 = b - Ax_0$ is the residual
of some initial guess $x_0$. $\mathcal{K}_m$ is related to the original problem
space by

$$
V_m^TAV_m = H_m
$$

where $V_m$ is an orthonormal basis of $\mathcal{K}_m$ and $H_m$ is a
$m \times m$ Hessenberg matrix. Arnoldi's method computes these matrices.

The benefit of this approach is that a problem based on $H_m$ is
smaller than the original problem (controllable by the choice of subspace dimension $m$)
and its Hessenberg structure makes it easier to solve with direct methods.
The tradeoff is that the smaller $m$ is, the less accurate the solution will be.

Getting an accurate result in one iteration of FOM requires working in a
high-dimensional Krylov subspace, which has a computational complexity of $O(m^2)$ due to
orthogonalization requiring dot products with all previously computed basis vectors.
Saad (2003) presents two variants of FOM to alleviate this, the Restarted and Incomplete versions.

### Restarted FOM

Restarted FOM simply runs FOM repeatedly with a small $m$ until a desired
precision is achieved.


In [2]:
def restarted_fom(eq: Equation, x0: np.ndarray, subsp_dim: int) -> RunResult:
    """Approximately solve `Ax = b` using the Full Orthogonalization Method
    with Krylov subspace dimension `subsp_dim`."""

    # Krylov subspace can't be larger than the column count of A
    if eq.A.shape[1] < subsp_dim:
        subsp_dim = eq.A.shape[1]

    # stop condition from Saad (2003): reduce residual norm by a factor of 10^7
    initial_resid_norm = np.linalg.norm(eq.residual(x0))
    stop_resid_limit = 1e-7 * initial_resid_norm
    x = np.copy(x0)
    iter_count = 0
    while iter_count < MAX_ITERATIONS:
        iter_count += 1
        # current residual
        resid = eq.residual(x)
        resid_norm = np.linalg.norm(resid)
        # stop if we're close enough to the correct answer
        # or the algorithm failed and diverged to infinity (this happens with FIDAP036)
        if resid_norm < stop_resid_limit or math.isinf(resid_norm):
            break
        # orthonormal basis of the Krylov subspace,
        # filled in over the course of the algorithm
        V = np.zeros((eq.A.shape[1], subsp_dim))
        # the Hessenberg matrix H in the relation (V^T)AV = H
        H = np.zeros((subsp_dim, subsp_dim))
        # first basis vector of the Krylov subspace based on the residual
        V[:,0] = resid / resid_norm

        for col in range(subsp_dim):
            # the next vector in the Krylov subspace's basis
            w = eq.A * V[:,col]
            # orthogonalize using Modified Gram-Schmidt
            for prev_col in range(col+1):
                H[prev_col, col] = w.dot(V[:,prev_col])
                w -= H[prev_col, col] * V[:,prev_col]
            if col+1 == subsp_dim:
                break
            H[col+1, col] = np.linalg.norm(w)
            if H[col+1, col] < EPSILON:
                # terminated early because the Krylov subspace's actual dimension
                # is less than what was given as parameter.
                # resize the matrices H and V so that they're not singular if this happens
                H = H[:col+1, :col+1]
                V = V[:, :col+1]
                break
            V[:,col+1] = w / H[col+1, col]

        # Using a prebuilt routine for solving the Hessenberg system for now.
        # I know Saad (2003) has a method for this, but that seems less relevant than
        # trying many different methods, so I'll leave it for later if I have time.
        h_rhs = np.zeros((H.shape[1], 1))
        h_rhs[0] = resid_norm
        y = np.linalg.solve(H, h_rhs)

        # y and its product with V are 2D column vectors,
        # but the rest of the code works in 1D vectors, hence the weird indexing here
        x += np.dot(V, y)[:,0]

    return RunResult(
        ans=x,
        iterations=iter_count,
    )

In [3]:
for dim in [10, 30, 50]:
    print(f"Subspace dimension: {dim}")
    test_run(restarted_fom, subsp_dim=dim)
    print("")

Subspace dimension: 10
     matrix    |      size     |  pre-residual | post-residual |   iterations  |   time (ms)   
---------------|---------------|---------------|---------------|---------------|---------------
    FIDAP005   |    27 x 27    |   2.751e+06   |   1.263e+01   |      300      |      178      
    FIDAP036   |  3079 x 3079  |   9.259e+02   |   1.015e+03   |      300      |      729      
     GR3030    |   900 x 900   |   7.439e+01   |   2.205e-06   |       14      |       11      
    ORSIRR1    |  1030 x 1030  |   5.394e+05   |   3.513e+00   |      300      |      444      

Subspace dimension: 30
     matrix    |      size     |  pre-residual | post-residual |   iterations  |   time (ms)   
---------------|---------------|---------------|---------------|---------------|---------------
    FIDAP005   |    27 x 27    |   1.433e+06   |   1.040e-09   |       2       |       3       
    FIDAP036   |  3079 x 3079  |   8.821e+02   |   1.943e+39   |      300      |      269

### Direct Incomplete Orthogonalization Method (DIOM)

The basic idea of incomplete orthogonalization is simple: reduce the cost
of orthogonalization by only comparing against the last $k$ computed basis vectors.
However, this only reduces computational costs while keeping memory costs
the same. With some additional work, a special LU factorization can be created
that does not need to store the entire basis $V_m$ in memory.
This is the Direct variant detailed in chapter 6.4.2 of Saad (2003).


In [4]:
from collections import deque

def direct_iom(eq: Equation, x0: np.ndarray, ortho_count: int) -> RunResult:
    """Approximately solve `Ax = b` using the Direct Incomplete
    Orthogonalization Method, orthogonalizing against `ortho_count` vectors."""

    initial_resid = eq.residual(x0)
    initial_resid_norm = np.linalg.norm(initial_resid)
    stop_resid_limit = 1e-7 * initial_resid_norm

    # the last `ortho_count` vectors of the basis of K_m.
    # deque automatically handles dropping values we don't need anymore
    V = deque([], ortho_count)
    # first basis vector from the initial residual
    V.append(initial_resid / initial_resid_norm)
    # P = VU^{-1}
    # one less entry because the `m`th vector is computed from `k-1` previous ones
    P = deque([], ortho_count - 1)
    # lower triangular part of the LU decomposition of H
    # one entry less because the final column does not have the subdiagonal
    L = deque([], ortho_count - 1)
    # zeta = L^{-1}(\beta e_1)
    # only one iteration of zeta needs to be stored
    zeta = 0
    # the rest of the state involved (H and U)
    # can be computed one vector at a time and don't need to be stored
    
    x = np.copy(x0)
    step_idx = 0
    # stop if we cover all of the dimension of A without converging
    while step_idx < eq.A.shape[1]:
        # check convergence
        resid_norm = np.linalg.norm(eq.residual(x))
        if resid_norm < stop_resid_limit or math.isinf(resid_norm):
            break

        # we don't actually need to know the indices where we are in the "full" matrix,
        # indexing into the incomplete arrays and vectors in the right way
        # gives us all the information we need

        next_v = eq.A * V[-1]
        # only store the nonzero entries of H, of which there are at most
        # `ortho_count + 1` because H is banded Hessenberg
        next_h = np.zeros(len(V) + 1)
        for col in range(len(V)):
            next_h[col] = next_v.dot(V[col])
            next_v -= next_h[col] * V[col]
        # entry of H below the main diagonal
        next_h[len(V)] = np.linalg.norm(next_v)
        next_v /= next_h[len(V)]
        # next_v is V_{m+1} which isn't used until next iteration,
        # so don't append it to V until the end of the loop

        if step_idx == 0:
            # first step is a special case where the subdiagonal entry of L does not exist yet
            zeta = initial_resid_norm
            next_u = np.copy(next_h[0:-1])
            # compute the L value that will be used in the next step now
            # so we don't have to store an additional column of H
            L.append(next_h[-1] / next_u[-1])
        else:
            # LU factorization based on notes below
            # once again only storing nonzero entries in `ortho_count`-vectors for U
            # and scalars for L.
            next_u = np.copy(next_h[0:-1])
            for i in range(1, len(next_u)):
                next_u[i] -= L[i-1] * next_u[i-1]

            if abs(next_u[-1]) < EPSILON:
                break

            zeta *= -L[-1]

            # again, this value of L will be used in the next step
            L.append(next_h[-1] / next_u[-1])

        next_p = (
            (1.0 / next_u[-1])
            * (V[-1] - sum([next_u[i] * P[i] for i in range(len(next_u) - 1)]))
        )

        P.append(next_p)

        V.append(next_v)

        x += zeta * next_p

        step_idx += 1

    return RunResult(
        ans=x,
        iterations=step_idx,
    )

In [5]:
for ortho_count in [5, 10, 50]:
    print(f"Orthogonal vectors: {ortho_count}")
    test_run(direct_iom, ortho_count=ortho_count)
    print("")

Orthogonal vectors: 5
     matrix    |      size     |  pre-residual | post-residual |   iterations  |   time (ms)   
---------------|---------------|---------------|---------------|---------------|---------------
    FIDAP005   |    27 x 27    |   3.733e+06   |   1.067e+02   |       27      |       4       
    FIDAP036   |  3079 x 3079  |   8.742e+02   |   5.727e-03   |      3079     |      785      
     GR3030    |   900 x 900   |   7.294e+01   |   6.709e-06   |       58      |       7       
    ORSIRR1    |  1030 x 1030  |   5.541e+05   |   2.157e+03   |      1030     |      132      

Orthogonal vectors: 10
     matrix    |      size     |  pre-residual | post-residual |   iterations  |   time (ms)   
---------------|---------------|---------------|---------------|---------------|---------------
    FIDAP005   |    27 x 27    |   3.015e+06   |   6.462e+01   |       27      |       4       
    FIDAP036   |  3079 x 3079  |   9.610e+02   |   3.436e-03   |      3079     |      976 

#### Updating the LU decomposition

Saad (2003) doesn't explicitly say how the LU decomposition of $H_m$ is computed,
so I'm going to try and figure it out myself. Say we have $k = 3$ and
at step $m = 10$ of the algorithm,

$$
H_{10} = \begin{bmatrix}
\ddots & \\
& h_{6,8} \\
& h_{7,8} & h_{7,9} \\
& h_{8,8} & h_{8,9} & h_{8,10} \\
& h_{9,8} & h_{9,9} & h_{9,10} \\
& & h_{10,9} & h_{10,10} \\
& & & h_{11,10} \\
\end{bmatrix}
$$

We've just computed the last column of $H_{10}$ and we also have the matrices

$$
L_{9} = \begin{bmatrix}
\ddots & \\
& l_{7,6} & 1 \\
& & l_{8,7} & 1 \\
& & & l_{9,8} & 1 \\
\end{bmatrix}
$$

and

$$
U_9 = \begin{bmatrix}
\ddots & \\
& u_{5,7} \\
& u_{6,7} & u_{6,8} \\
& u_{7,7} & u_{7,8} & u_{7,9} \\
& & u_{8,8} & u_{8,9} \\
& & & u_{9,9} \\
\end{bmatrix}.
$$

For the second to last subdiagonal entry of $H_{10}$ which we haven't yet considered
in the LU decomposition, we have

$$
h_{10,9} = l_{10,9}u_{9,9}
$$

from which we get the new subdiagonal entry of $L_{10}$

$$
l_{10,9} = \frac{h_{10,9}}{u_{9,9}}.
$$

Additionally the equations for the final column of $L_{10}U_{10} = H_{10}$ are

$$
\begin{align*}
h_{8,10} &= l_{8,7}u_{7,10} + u_{8,10} = u_{8,10} \iff u_{8,10} = h_{8,10} \\
h_{9,10} &= l_{9,8}u_{8,10} + u_{9,10} \iff u_{9,10} = h_{9,10} - l_{9,8}u_{8,10} \\
h_{10,10} &= l_{10,9}u_{9,10} + u_{10,10} \iff u_{10,10} = h_{10,10} - l_{10,9}u_{9,10} \\
\end{align*}
$$

From this we can deduce an iterative algorithm for the elements of $u_{*,10}$.

$$
\begin{align*}
u_{m-k,m} &= h_{m-k,m} \\
u_{i,m} &= h_{i,m} - l_{i,i-1}u_{i-1,m} \text{ for } i = m-k+1, \dots, m \\
\end{align*}
$$

### GMRES


In [59]:
def restarted_gmres(eq: Equation, x0: np.ndarray, subsp_dim: int) -> RunResult:
    """Approximately solve `Ax = b` using the Generalized Minimum Residual Method
    with Krylov subspace dimension `subsp_dim`."""

    # Krylov subspace can't be larger than the column count of A
    if eq.A.shape[1] < subsp_dim:
        subsp_dim = eq.A.shape[1]

    # stop condition from Saad (2003): reduce residual norm by a factor of 10^7
    initial_resid_norm = np.linalg.norm(eq.residual(x0))
    stop_resid_limit = 1e-7 * initial_resid_norm
    # flag needed to break the outer iteration from an inner loop
    has_converged = False
    x = np.copy(x0)
    iter_count = 0
    while not has_converged and iter_count < MAX_ITERATIONS:
        iter_count += 1
        resid = eq.residual(x)
        resid_norm = np.linalg.norm(resid)
        # matrices stored as arrays, built incrementally
        # first basis vector of the Krylov subspace based on the residual
        V = [resid / resid_norm]
        # the upper diagonal form of the Hessenberg matrix H
        R = []
        # `g` in the least-squares problem `argmin ||g - Hy||`
        rhs = [resid_norm]
        # `s` and `c` in the rotation matrices used transform H into upper triangular form
        rot_s = []
        rot_c = []

        for col in range(subsp_dim):
            # the next vector in the Krylov subspace's basis
            next_v = eq.A * V[-1]
            # vector of the nonzero entries of the next column of R.
            # we first compute the corresponding column of H into this vector
            # and then apply rotations to arrive at R
            next_r = np.zeros(col+1)
            # orthogonalize
            for prev_col in range(len(V)):
                next_r[prev_col] = next_v.dot(V[prev_col])
                next_v -= next_r[prev_col] * V[prev_col]
            # the entry of H below the diagonal, which we will eliminate with a rotation later
            h_subdiag = np.linalg.norm(next_v)

            # apply previous plane rotations to the new column of H
            for row in range(len(rot_s)):
                h1 = rot_c[row] * next_r[row] + rot_s[row] * next_r[row+1]
                h2 = -rot_s[row] * next_r[row] + rot_c[row] * next_r[row+1]
                next_r[row] = h1
                next_r[row+1] = h2

            # next plane rotation matrix
            denom = math.sqrt(next_r[-1]**2 + h_subdiag**2)
            next_s = h_subdiag / denom
            next_c = next_r[-1] / denom
            # left-multiply H by it
            r_diag = next_c * next_r[-1] + next_s * h_subdiag
            # debug: check that this indeed eliminates the subdiagonal entry of H
            assert(abs(-next_s * next_r[-1] + next_c * h_subdiag) < EPSILON)
            next_r[-1] = r_diag
            R.append(next_r)
            # also left-multiply rhs by it (after appending a zero to it)
            next_rhs = -next_s * rhs[-1]
            rhs[-1] = next_c * rhs[-1]
            # the absolute value of `next_rhs` is the residual norm at this step.
            # if it's low enough, we can stop and move on to solving the triangular system
            if abs(next_rhs) < stop_resid_limit:
                has_converged = True
                break

            rot_s.append(next_s)
            rot_c.append(next_c)
            if col+1 < subsp_dim:
                rhs.append(next_rhs)
                V.append(next_v / h_subdiag)

        # backwards substitution to solve the upper triangular system `Ry = g`
        y = np.zeros(len(R))
        for row in reversed(range(len(y))):
            y[row] = (
                (1.0 / R[row][row])
                * (rhs[row]
                    - sum([R[col][row] * y[col] for col in range(row + 1, len(y))]))
            )

        V = np.array(V).T
        x += V @ y

    return RunResult(
        ans=x,
        iterations=iter_count,
    )

In [60]:
for dim in [10, 30, 50]:
    print(f"Subspace dimension: {dim}")
    test_run(restarted_gmres, subsp_dim=dim)
    print("")

Subspace dimension: 10
     matrix    |      size     |  pre-residual | post-residual |   iterations  |   time (ms)   
---------------|---------------|---------------|---------------|---------------|---------------
    FIDAP005   |    27 x 27    |   2.485e+06   |   9.987e-01   |      300      |      224      
    FIDAP036   |  3079 x 3079  |   9.077e+02   |   4.020e-01   |      300      |      640      
     GR3030    |   900 x 900   |   7.748e+01   |   7.623e-06   |       15      |       13      
    ORSIRR1    |  1030 x 1030  |   5.766e+05   |   8.232e-01   |      300      |      490      

Subspace dimension: 30
     matrix    |      size     |  pre-residual | post-residual |   iterations  |   time (ms)   
---------------|---------------|---------------|---------------|---------------|---------------
    FIDAP005   |    27 x 27    |   1.943e+06   |   1.876e-01   |       1       |       3       
    FIDAP036   |  3079 x 3079  |   9.250e+02   |   3.970e-03   |      300      |      231

# Sources

Saad, Y. (2003). Iterative Methods for Sparse Linear Systems.